In [1]:
#####################################
#Econs 514 -- Assignment 1
#Updated -- 1/30/23
#By -- Suhina Deol
#####################################

#Import packages
import os
import numpy as np
import pandas as pd
import scipy.optimize as opt
from numpy.random import randn

In [2]:
#####################################
#load data as usual
#####################################
df = pd.read_csv(r'iCloudDrive/Documents/Econs 514 (Metrics IV)/Assignment1/assignment1_data.csv')
df['cons'] = 1.
df['Alt'] = range(len(df))
print(df)

#Variables
# id, route, toll, median, femaile, age3050, distance, householdsize
# high_income, med_income,occupanc, trans


          id  route   toll    median  female  age3050  distance  \
0    1000169      1  3.250 -4.152752       1        1     34.58   
1    1000187      0  3.250 -4.849391       0        1     26.44   
2    1000201      1  1.625 -4.849391       0        1     24.09   
3    1000209      0  2.900 -3.243314       0        0     26.46   
4    1000282      1  2.900 -2.246180       0        1     45.88   
..       ...    ...    ...       ...     ...      ...       ...   
433  5004374      0  1.450 -2.249893       0        1     27.97   
434  5006382      0  1.950 -1.206373       1        0     33.47   
435  5006513      0  2.900 -2.595311       0        1     44.56   
436  5006568      0  2.900 -2.422315       0        0     61.02   
437  5011583      0  2.900 -1.165325       0        0     63.60   

     householdsize  high_income  med_income  occupanc  trans  cons  Alt  
0                2            1           0         1      1   1.0    0  
1                1            0           0    

In [84]:
class ChoiceModels(object):
    
    '''
    This class defines methods that will be used later in speficying and estimating choice models.
    '''
   
    def load_data(self, path, file):
        df = pd.read_csv(os.path.join(path, file), sep='\s+', header=0)
        df['cons'] = 1.
        return df
    
    def expand_data(self, df, n):
        '''
        n : Integer
            Number of times to expand the data
        Returns
        -------
        An expanded pandas data frame with a panel structure
        '''
        df['Alt'] = [[str(i) for i in range(n)] for _ in range(len(df))]
        return df.explode('Alt')
    
    def create_choice_attributes(self, df, config):
        '''
        This method creates a panel structure of data to estimate the multinomial
        choice model speficied in the configuration file (config-- a json format file)
        '''
        # create dependent variable
        y_namelist = list(config['Alternatives']['0'].keys())
        df['choice'] = list(zip(*[df[v] for v in y_namelist]))
        df = self.expand_data(df, len(config['Alternatives']))
   
        df['y'] = 0.
        for k,v in config['Alternatives'].items():
            label = tuple(v.values())
            df.loc[(df["Alt"]==k) & (df['choice']==label), 'y'] = 1
        
        # create alternative specific attributes
        dic = config['Attributes']
        for var,info in dic.items():
            df[var] = 0
            for alt, w in info['rule'].items():
                df['junk'] = 0
                df.loc[(df['Alt'] == alt), 'junk'] = 1
                df[var] = df[var] + w * df[info['variable']] * df['junk'] 
        df = df.drop("junk", axis='columns')
        
        # create interactions
        df, xz_list = self.create_interactions(df, config['Interactions']) 
        x_list = list(config['Attributes'].keys()) + xz_list
        return {'data': df, "var_names": x_list}
    
    
    def create_interactions(self, df, interact_list):
        '''
        interact_list : a List
            The list contains pairs of variable names as tuples
        Returns
        -------
        df : pandas data frame after adding interactions
            
        xz_list : A list of created interactions
        '''
        xz_list = []
        for item in interact_list:
            vname = item[0] + "_" + item[1]
            df[vname] = df[item[0]] * df[item[1]]
            xz_list.append(vname)
        return df, xz_list 
        
        
    def optimization(self, objfun, para):
        '''
        Parameters
        ----------
        objfun : a user defined objective function of para
            
        para : a 1-D array with the shape (k,), where k is the number of parameters.
        Returns
        -------
        dict
            A dictionary containing estimation results
        '''
        v = opt.minimize(objfun, x0=para, jac=None, method='BFGS', 
                          options={'maxiter': 1000, 'disp': True})  
        return {'log_likelihood':-1*v.fun, "Coefficients": v.x, "Var_Cov": v.hess_inv}

In [5]:
class BinaryLogit(ChoiceModels):
    '''
    This class is to estimate a binary logit nodel by MLE.  
    '''
    def __init__(self, path, file, yname, x=None, z=None, interactions=None):
        df = super().load_data(path, file)
        if x is None:
            x = []
        if z is None:
            z = []
        if interactions is None:
            xz = []
            self.df = df
        else:
            self.df, xz = super().create_interactions(df, interactions)
            
        self.X_list = ['cons'] + x + z + xz
        self.Xmat = self.df[self.X_list].to_numpy()
        self.y = self.df[yname].to_numpy()
        
    def log_likelihood(self, para):
        '''
        Returns
        -------
        res : scalar
            log-likelihood value

        '''
        xb = np.matmul(self.Xmat, para)
        xb = np.exp(xb)
        xb = xb / (1+xb) #sjt
        return (-1/len(xb)) * np.sum(self.y * np.log(xb) + (1-self.y) * np.log(1 - xb))
   
    def estimation(self, para):
        '''
        Returns
        -------
        A dictionary of estimation results
        '''
        return super().optimization(self.log_likelihood, para)


In [6]:
#####################################
#Question 1 
#Develop a binary choice model on commuters’ transponder acquisition decision.
#####################################

if __name__ == '__main__':
    path = r"iCloudDrive/Documents/Econs 514 (Metrics IV)/Assignment1"
    file = "assignment1_data.txt"
    
    ## estimating binary models
    x = ['toll', 'median']
    z = ['female', 'age3050', 'householdsize']
    interactions = [('toll', 'high_income'), ('toll', 'med_income')]
    trans = BinaryLogit(path, file, "trans", x=x, z=z, interactions=interactions)
    bini = np.zeros(len(trans.X_list))
    res_binary = trans.estimation(bini)

print(" ")
print("Question 1: Y=Trans")
print(" ")
print(res_binary)

Optimization terminated successfully.
         Current function value: 0.595534
         Iterations: 32
         Function evaluations: 306
         Gradient evaluations: 34
 
Question 1: Y=Trans
 
{'log_likelihood': -0.5955342602669939, 'Coefficients': array([-0.36607586, -0.31970981, -0.18845208,  0.60599558,  0.8099458 ,
       -0.127574  ,  0.66246452,  0.41293508]), 'Var_Cov': array([[ 1.13953640e+02, -3.26341347e+01, -2.77850586e+00,
        -7.34637869e+00, -1.06898544e+01, -6.70605752e+00,
        -1.50369949e+00,  5.14933615e-01],
       [-3.26341347e+01,  1.64607423e+01,  4.03091620e+00,
         1.45874617e+00,  1.18015099e+00,  5.87001593e-01,
        -7.61627656e-01, -1.28664327e+00],
       [-2.77850586e+00,  4.03091620e+00,  2.88392988e+00,
         8.49730863e-01, -8.31363069e-02,  1.84063151e-01,
        -1.74198482e-01, -4.50394933e-03],
       [-7.34637869e+00,  1.45874617e+00,  8.49730863e-01,
         1.87999815e+01,  1.88843943e+00, -3.22073077e-01,
         5.6494

In [112]:
#check - https://www.pythonfordatascience.org/logistic-regression-python/
import statsmodels.formula.api as smf

df['toll_high']=df['toll']*df['high_income']
df['toll_med']=df['toll']*df['med_income']

q1_check = smf.logit("trans ~ toll + median + female + age3050 + householdsize + toll_high + toll_med", data = df).fit()

q1_check.summary()



Optimization terminated successfully.
         Current function value: 0.595534
         Iterations 5


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                  trans   No. Observations:                  438
Model:                          Logit   Df Residuals:                      430
Method:                           MLE   Df Model:                            7
Date:                Tue, 31 Jan 2023   Pseudo R-squ.:                  0.1206
Time:                        21:30:04   Log-Likelihood:                -260.84
converged:                       True   LL-Null:                       -296.62
Covariance Type:            nonrobust   LLR p-value:                 7.198e-13
=================================================================================
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept        -0.3661      0.542     -0.676      0.499      -1.428       0.696
toll             -0.3197      0.210     -1.519      0.129      -0.732       0.093
median           -0.1885      0.087     -2.165      0.030      -0.359      -0.018
female            0.6060      0.235      2.579      0.010       0.145       1.066
age3050           0.8100      0.224      3.614      0.000       0.371       1.249
householdsize    -0.1276      0.072     -1.766      0.077      -0.269       0.014
toll_high         0.6625      0.115      5.764      0.000       0.437       0.888
toll_med          0.4129      0.087      4.772      0.000       0.243       0.583
=================================================================================
"""

In [21]:
class MultinomialLogit(ChoiceModels):

    # Specify model here    
    model_config = {"Alternatives":
                    {"0": {"occupanc": 1, "route": 1},
                     "1": {"occupanc": 1, "route": 0},
                     "2": {"occupanc": 2, "route": 1},
                     "3": {"occupanc": 2, "route": 0},
                     "4": {"occupanc": 3, "route": 1},
                     "5": {"occupanc": 3, "route": 0}},
                    "Attributes":{'express_dummy':{'variable':'cons', 
                                                   'rule':{"0":1,"2":1,"4":1}},
                                  'hov2_dummy':{'variable':'cons', 
                                               'rule':{"2":1,"3":1}},
                                  "hov3_dummy":{'variable':'cons', 
                                                'rule':{"4":1,"5":1}},
                                  "price":{"variable": 'toll', 
                                           "rule": {"0":1,"2":1/2,"4":1/6}},
                                  "time": {"variable":"median", 
                                           "rule":{"0":1,"2":1,"4":1}}},
                    "Interactions":[('price', "high_income"), ('price', "med_income"),
                                    ("hov2_dummy", "householdsize"),
                                    ("hov3_dummy", "householdsize")]}  
    
    def __init__(self, path, file):
        df = super().load_data(path, file)
        res = super().create_choice_attributes(df, MultinomialLogit.model_config)
        self.df = res['data']
        self.X_list = res['var_names']
        self.y = self.df['y'].to_numpy()
        self.Xmat = self.df[self.X_list].to_numpy()
        

               
    def mnl_log_likelihood(self, para):
        '''
        This method defines the data log-likelihood from a Multinomial Logit.
        '''
        df = self.df.copy()
        xb = np.matmul(self.Xmat, para)
        xb = np.exp(xb)
        df['xb'] = xb.tolist()
        
        # group sum
        df['xbsum'] = df.groupby(['id'])["xb"].transform(lambda x: x.sum())
        df['log_likelihood'] = df['y']*np.log(df['xb'] / df['xbsum'])
        return (-1/len(df))* np.sum(df['log_likelihood'])
 
    def estimation(self, para):
        '''
        Parameters
        ----------
        para : array
            a 1-D array with the shape(k,), where k is the number of model parameters.

        Returns
        -------
        A dictionary of estimation results
        '''
        return super().optimization(self.mnl_log_likelihood, para)
    

In [22]:
#####################################
#Question 2
#Develop a multinomial choice model on commuters joint occ/route
#####################################

if __name__ == '__main__':
    path = r"iCloudDrive/Documents/Econs 514 (Metrics IV)/Assignment1"
    file = "assignment1_data.txt"
    
    ## estimating a MNL model
    mnl = MultinomialLogit(path, file)
    bini = np.zeros(len(mnl.X_list))
    res_mnl = mnl.estimation(bini)
    
print(" ")
print("Question 2: Y=Occup, Route")
print(" ")
print(res_mnl)

Optimization terminated successfully.
         Current function value: 0.221286
         Iterations: 54
         Function evaluations: 550
         Gradient evaluations: 55
 
Question 2: Y=Occup, Route
 
{'log_likelihood': -0.22128648216886576, 'Coefficients': array([-0.88781567, -1.25891228, -2.96283412, -0.4990075 , -0.15584211,
        0.46693719,  0.20726772, -0.01782022,  0.17530502]), 'Var_Cov': array([[ 2.27899622e+02, -3.36754857e+01, -5.79812950e+01,
        -5.43229334e+01,  3.07847216e+01,  1.15939256e+01,
         3.44012628e+00,  2.43455578e+00,  1.51951180e+00],
       [-3.36754857e+01,  2.11694985e+02,  1.13208711e+02,
         2.04586348e+01,  2.88133381e+00, -5.25914722e-01,
        -4.03391664e+00, -4.94463371e+01, -2.35635379e+01],
       [-5.79812950e+01,  1.13208711e+02,  4.55298209e+02,
         3.04472851e+01,  2.71671989e+00, -6.00135801e-01,
        -3.23106498e+00, -2.50320203e+01, -9.94360467e+01],
       [-5.43229334e+01,  2.04586348e+01,  3.04472851e+01,
  

In [40]:
class NestedLogit(ChoiceModels):

    # Specify model here    
    model_config = {"Alternatives":
                    {"0": {"trans": 1, "occupanc": 1, "route": 1},
                     "1": {"trans": 1, "occupanc": 1, "route": 0},
                     "2": {"trans": 1, "occupanc": 2, "route": 1},
                     "3": {"trans": 1, "occupanc": 2, "route": 0},
                     "4": {"trans": 1, "occupanc": 3, "route": 1},
                     "5": {"trans": 1, "occupanc": 3, "route": 0},
                     "6": {"trans": 0, "occupanc": 1, "route": 0},
                     "7": {"trans": 0, "occupanc": 2, "route": 0},
                     "8": {"trans": 0, "occupanc": 3, "route": 0}},
                    "Nests": {"0":{"0": ["0", "1"], "1": ["2", "3"], 
                                   "2": ["4", "5"]},"1":["6", "7", "8"]},
                    "Attributes":{'trans_dummy':{'variable': 'cons', 
                                                 'rule':{"0":1,"1":1,
                                                         "2":1,"3":1,"4":1,"5":1}},
                                  'express_dummy':{'variable':'cons', 
                                                   'rule':{"0":1,"2":1,"4":1}},
                                  'hov2_dummy':{'variable':'cons', 
                                               'rule':{"2":1,"3":1,"7":1}},
                                  "hov3_dummy":{'variable':'cons', 
                                                'rule':{"4":1,"5":1,"8":1}},
                                  "price":{"variable": 'toll', 
                                           "rule": {"0":1,"2":1/2,"4":1/6}},
                                  "time": {"variable":"median", 
                                           "rule":{"0":1,"2":1,"4":1}}},
                    "Interactions":[('price', "high_income"), ('price', "med_income"),
                                    ("hov2_dummy", "householdsize"),
                                    ("hov3_dummy", "householdsize")]}
    
    
    def __init__(self, path, file):
        df = super().load_data(path, file)
        res = super().create_choice_attributes(df, NestedLogit.model_config)
        self.df = res['data']
        self.X_list = res['var_names']
        self.y = self.df['y'].to_numpy()
        self.Xmat = self.df[self.X_list].to_numpy()
        
               
    def log_likelihood(self, para):
        df = self.df.copy()
        xb = np.matmul(self.Xmat, para)
        l = np.exp(xb)
        xb = np.exp(xb/l)
        df['xb'] = xb.tolist()
        

        # group sum
        df['xbsum'] = df.groupby(['id'])["xb"].transform(lambda x: x.sum())
        df['nom_1'] = df['xbsum']**(l-1)
        df['nom_2'] = df['xb']*df['nom_1']
        df['denom_1'] = df['xbsum']**l
        df['denom_2']  = np.sum(df['denom_1'])
        df['log_likelihood'] = df['y']*np.log(df['nom_2']/df['denom_2'])
        return (-1/len(df))*np.sum(df['log_likelihood'])
 
    def estimation(self, para):
        return super().optimization(self.log_likelihood, para)


In [41]:
#####################################
#Question 3 
#Develop a nested-logit model in which commuters first choose to 
#acquire trans, then car occupancy, then road
#####################################

if __name__ == '__main__':
    path = r"iCloudDrive/Documents/Econs 514 (Metrics IV)/Assignment1"
    file = "assignment1_data.txt"
    
    nl = NestedLogit(path, file)
    bini = np.zeros(len(nl.X_list))
    res_nl = nl.estimation(bini)
    
print(" ")
print("Question 3: Y=Trans,Occu,Route")
print(" ")
print(res_nl)

Optimization terminated successfully.
         Current function value: 1.099421
         Iterations: 36
         Function evaluations: 429
         Gradient evaluations: 39
 
Question 3: Y=Trans,Occu,Route
 
{'log_likelihood': -1.0994213113063216, 'Coefficients': array([ 0.07711654, -0.08064035, -0.60432807, -1.12793525, -0.19948298,
       -0.05265958,  0.19875186,  0.09547242, -0.01648087,  0.0409737 ]), 'Var_Cov': array([[ 7.64278998e+00, -3.61638678e+00, -1.87655921e+00,
        -2.29730722e+00, -6.59441632e-01,  8.74745834e-02,
        -1.80374899e-01,  3.54707824e-01, -1.31795783e-01,
        -1.63049371e-01],
       [-3.61638678e+00,  3.08427089e+01, -1.17563904e+00,
        -2.41155207e+00, -6.39127150e+00,  4.05273246e+00,
         8.66061398e-01, -3.39221891e-01, -5.30581727e-02,
        -2.55382282e-01],
       [-1.87655921e+00, -1.17563904e+00,  2.58506359e+01,
         3.24012448e+00,  8.13481579e-01, -1.56258844e-01,
        -3.99056854e-01, -1.11084531e-01, -5.61499754e+

In [42]:
#specifying l =0.5 here to test results from Question 3
class NestedLogit(ChoiceModels):

    # Specify model here    
    model_config = {"Alternatives":
                    {"0": {"trans": 1, "occupanc": 1, "route": 1},
                     "1": {"trans": 1, "occupanc": 1, "route": 0},
                     "2": {"trans": 1, "occupanc": 2, "route": 1},
                     "3": {"trans": 1, "occupanc": 2, "route": 0},
                     "4": {"trans": 1, "occupanc": 3, "route": 1},
                     "5": {"trans": 1, "occupanc": 3, "route": 0},
                     "6": {"trans": 0, "occupanc": 1, "route": 0},
                     "7": {"trans": 0, "occupanc": 2, "route": 0},
                     "8": {"trans": 0, "occupanc": 3, "route": 0}},
                    "Nests": {"0":{"0": ["0", "1"], "1": ["2", "3"], 
                                   "2": ["4", "5"]},"1":["6", "7", "8"]},
                    "Attributes":{'trans_dummy':{'variable': 'cons', 
                                                 'rule':{"0":1,"1":1,
                                                         "2":1,"3":1,"4":1,"5":1}},
                                  'express_dummy':{'variable':'cons', 
                                                   'rule':{"0":1,"2":1,"4":1}},
                                  'hov2_dummy':{'variable':'cons', 
                                               'rule':{"2":1,"3":1,"7":1}},
                                  "hov3_dummy":{'variable':'cons', 
                                                'rule':{"4":1,"5":1,"8":1}},
                                  "price":{"variable": 'toll', 
                                           "rule": {"0":1,"2":1/2,"4":1/6}},
                                  "time": {"variable":"median", 
                                           "rule":{"0":1,"2":1,"4":1}}},
                    "Interactions":[('price', "high_income"), ('price', "med_income"),
                                    ("hov2_dummy", "householdsize"),
                                    ("hov3_dummy", "householdsize")]}
    
    
    def __init__(self, path, file):
        df = super().load_data(path, file)
        res = super().create_choice_attributes(df, NestedLogit.model_config)
        self.df = res['data']
        self.X_list = res['var_names']
        self.y = self.df['y'].to_numpy()
        self.Xmat = self.df[self.X_list].to_numpy()
        
               
    def log_likelihood(self, para):
        df = self.df.copy()
        xb = np.matmul(self.Xmat, para)
        l = 0.5
        xb = np.exp(xb/l)
        df['xb'] = xb.tolist()
        

        # group sum
        df['xbsum'] = df.groupby(['id'])["xb"].transform(lambda x: x.sum())
        df['nom_1'] = df['xbsum']**(l-1)
        df['nom_2'] = df['xb']*df['nom_1']
        df['denom_1'] = df['xbsum']**l
        df['denom_2']  = np.sum(df['denom_1'])
        df['log_likelihood'] = df['y']*np.log(df['nom_2']/df['denom_2'])
        return (-1/len(df))*np.sum(df['log_likelihood'])
 
    def estimation(self, para):
        return super().optimization(self.log_likelihood, para)

if __name__ == '__main__':
    path = r"iCloudDrive/Documents/Econs 514 (Metrics IV)/Assignment1"
    file = "assignment1_data.txt"
    
    nl = NestedLogit(path, file)
    bini = np.zeros(len(nl.X_list))
    res_nl = nl.estimation(bini)
    
print(" ")
print("Question 3: Y=Trans,Occu,Route")
print(" ")
print(res_nl)

Optimization terminated successfully.
         Current function value: 1.123897
         Iterations: 44
         Function evaluations: 495
         Gradient evaluations: 45
 
Question 3: Y=Trans,Occu,Route
 
{'log_likelihood': -1.1238965702363601, 'Coefficients': array([-0.11506893, -0.02732602, -0.6202826 , -1.47213572, -0.23992951,
       -0.07206926,  0.21502084,  0.09809429, -0.01131114,  0.08562851]), 'Var_Cov': array([[ 9.81819239e+00, -6.14630392e+00, -8.86734350e-01,
         3.75235587e-01, -1.83488030e-01, -2.28454053e-01,
         1.81442206e-01,  2.34333919e-02,  3.09330187e-01,
        -5.09633634e-02],
       [-6.14630392e+00,  9.35516403e+01, -3.65674750e+00,
        -2.21628027e+01, -2.38695057e+01,  1.04396523e+01,
         2.86226041e+00,  2.52256066e+00, -1.22160703e+00,
         8.69662570e-01],
       [-8.86734350e-01, -3.65674750e+00,  7.11646568e+01,
         2.73947869e+01,  4.81924010e+00,  1.72738305e+00,
        -2.17473576e+00, -8.70028511e-01, -1.69267818e+

In [43]:
#specifying l =0.9 here to test results from Question 3
class NestedLogit(ChoiceModels):

    # Specify model here    
    model_config = {"Alternatives":
                    {"0": {"trans": 1, "occupanc": 1, "route": 1},
                     "1": {"trans": 1, "occupanc": 1, "route": 0},
                     "2": {"trans": 1, "occupanc": 2, "route": 1},
                     "3": {"trans": 1, "occupanc": 2, "route": 0},
                     "4": {"trans": 1, "occupanc": 3, "route": 1},
                     "5": {"trans": 1, "occupanc": 3, "route": 0},
                     "6": {"trans": 0, "occupanc": 1, "route": 0},
                     "7": {"trans": 0, "occupanc": 2, "route": 0},
                     "8": {"trans": 0, "occupanc": 3, "route": 0}},
                    "Nests": {"0":{"0": ["0", "1"], "1": ["2", "3"], 
                                   "2": ["4", "5"]},"1":["6", "7", "8"]},
                    "Attributes":{'trans_dummy':{'variable': 'cons', 
                                                 'rule':{"0":1,"1":1,
                                                         "2":1,"3":1,"4":1,"5":1}},
                                  'express_dummy':{'variable':'cons', 
                                                   'rule':{"0":1,"2":1,"4":1}},
                                  'hov2_dummy':{'variable':'cons', 
                                               'rule':{"2":1,"3":1,"7":1}},
                                  "hov3_dummy":{'variable':'cons', 
                                                'rule':{"4":1,"5":1,"8":1}},
                                  "price":{"variable": 'toll', 
                                           "rule": {"0":1,"2":1/2,"4":1/6}},
                                  "time": {"variable":"median", 
                                           "rule":{"0":1,"2":1,"4":1}}},
                    "Interactions":[('price', "high_income"), ('price', "med_income"),
                                    ("hov2_dummy", "householdsize"),
                                    ("hov3_dummy", "householdsize")]}
    
    
    def __init__(self, path, file):
        df = super().load_data(path, file)
        res = super().create_choice_attributes(df, NestedLogit.model_config)
        self.df = res['data']
        self.X_list = res['var_names']
        self.y = self.df['y'].to_numpy()
        self.Xmat = self.df[self.X_list].to_numpy()
        
               
    def log_likelihood(self, para):
        df = self.df.copy()
        xb = np.matmul(self.Xmat, para)
        l = 0.9
        xb = np.exp(xb/l)
        df['xb'] = xb.tolist()
        

        # group sum
        df['xbsum'] = df.groupby(['id'])["xb"].transform(lambda x: x.sum())
        df['nom_1'] = df['xbsum']**(l-1)
        df['nom_2'] = df['xb']*df['nom_1']
        df['denom_1'] = df['xbsum']**l
        df['denom_2']  = np.sum(df['denom_1'])
        df['log_likelihood'] = df['y']*np.log(df['nom_2']/df['denom_2'])
        return (-1/len(df))*np.sum(df['log_likelihood'])
 
    def estimation(self, para):
        return super().optimization(self.log_likelihood, para)

if __name__ == '__main__':
    path = r"iCloudDrive/Documents/Econs 514 (Metrics IV)/Assignment1"
    file = "assignment1_data.txt"
    
    nl = NestedLogit(path, file)
    bini = np.zeros(len(nl.X_list))
    res_nl = nl.estimation(bini)
    
print(" ")
print("Question 3: Y=Trans,Occu,Route")
print(" ")
print(res_nl)

Optimization terminated successfully.
         Current function value: 1.124203
         Iterations: 62
         Function evaluations: 693
         Gradient evaluations: 63
 
Question 3: Y=Trans,Occu,Route
 
{'log_likelihood': -1.1242027371974914, 'Coefficients': array([-0.2070733 , -0.0283958 , -1.08884913, -2.61837957, -0.39919696,
       -0.11289843,  0.33252742,  0.15716535, -0.02726249,  0.14773518]), 'Var_Cov': array([[ 3.19851309e+01, -1.50413832e+01, -1.99945394e+00,
         4.18059646e-01, -2.73569785e+00, -4.68872871e-01,
         5.85871912e-01,  2.69518053e-01, -2.40032049e-01,
        -5.09375147e-01],
       [-1.50413832e+01,  2.44688745e+02,  5.68445500e+00,
        -1.52447698e+01, -5.69003292e+01,  2.48669877e+01,
         1.76931754e+00, -4.18406156e-02, -4.60057382e+00,
        -4.78473103e+00],
       [-1.99945394e+00,  5.68445500e+00,  1.92706313e+02,
         2.18800128e+01,  2.56003150e+00,  3.83054413e+00,
         2.81353411e+00,  2.31733806e+00, -4.60197633e+

In [44]:
#specifying l =0.1 here to test results from Question 3
class NestedLogit(ChoiceModels):

    # Specify model here    
    model_config = {"Alternatives":
                    {"0": {"trans": 1, "occupanc": 1, "route": 1},
                     "1": {"trans": 1, "occupanc": 1, "route": 0},
                     "2": {"trans": 1, "occupanc": 2, "route": 1},
                     "3": {"trans": 1, "occupanc": 2, "route": 0},
                     "4": {"trans": 1, "occupanc": 3, "route": 1},
                     "5": {"trans": 1, "occupanc": 3, "route": 0},
                     "6": {"trans": 0, "occupanc": 1, "route": 0},
                     "7": {"trans": 0, "occupanc": 2, "route": 0},
                     "8": {"trans": 0, "occupanc": 3, "route": 0}},
                    "Nests": {"0":{"0": ["0", "1"], "1": ["2", "3"], 
                                   "2": ["4", "5"]},"1":["6", "7", "8"]},
                    "Attributes":{'trans_dummy':{'variable': 'cons', 
                                                 'rule':{"0":1,"1":1,
                                                         "2":1,"3":1,"4":1,"5":1}},
                                  'express_dummy':{'variable':'cons', 
                                                   'rule':{"0":1,"2":1,"4":1}},
                                  'hov2_dummy':{'variable':'cons', 
                                               'rule':{"2":1,"3":1,"7":1}},
                                  "hov3_dummy":{'variable':'cons', 
                                                'rule':{"4":1,"5":1,"8":1}},
                                  "price":{"variable": 'toll', 
                                           "rule": {"0":1,"2":1/2,"4":1/6}},
                                  "time": {"variable":"median", 
                                           "rule":{"0":1,"2":1,"4":1}}},
                    "Interactions":[('price', "high_income"), ('price', "med_income"),
                                    ("hov2_dummy", "householdsize"),
                                    ("hov3_dummy", "householdsize")]}
    
    
    def __init__(self, path, file):
        df = super().load_data(path, file)
        res = super().create_choice_attributes(df, NestedLogit.model_config)
        self.df = res['data']
        self.X_list = res['var_names']
        self.y = self.df['y'].to_numpy()
        self.Xmat = self.df[self.X_list].to_numpy()
        
               
    def log_likelihood(self, para):
        df = self.df.copy()
        xb = np.matmul(self.Xmat, para)
        l = 0.1
        xb = np.exp(xb/l)
        df['xb'] = xb.tolist()
        

        # group sum
        df['xbsum'] = df.groupby(['id'])["xb"].transform(lambda x: x.sum())
        df['nom_1'] = df['xbsum']**(l-1)
        df['nom_2'] = df['xb']*df['nom_1']
        df['denom_1'] = df['xbsum']**l
        df['denom_2']  = np.sum(df['denom_1'])
        df['log_likelihood'] = df['y']*np.log(df['nom_2']/df['denom_2'])
        return (-1/len(df))*np.sum(df['log_likelihood'])
 
    def estimation(self, para):
        return super().optimization(self.log_likelihood, para)

if __name__ == '__main__':
    path = r"iCloudDrive/Documents/Econs 514 (Metrics IV)/Assignment1"
    file = "assignment1_data.txt"
    
    nl = NestedLogit(path, file)
    bini = np.zeros(len(nl.X_list))
    res_nl = nl.estimation(bini)
    
print(" ")
print("Question 3: Y=Trans,Occu,Route")
print(" ")
print(res_nl)

Optimization terminated successfully.
         Current function value: 1.123727
         Iterations: 19
         Function evaluations: 264
         Gradient evaluations: 24
 
Question 3: Y=Trans,Occu,Route
 
{'log_likelihood': -1.1237266085392377, 'Coefficients': array([-0.02301138, -0.00723803, -0.12571907, -0.29637832, -0.04982406,
       -0.01553838,  0.04653942,  0.02068539, -0.0018283 ,  0.01754578]), 'Var_Cov': array([[ 5.05275675e-01, -2.81498651e-01,  9.67673246e-03,
         1.54246187e-02,  9.41302632e-03,  1.60085717e-02,
         2.70306904e-02, -6.60764745e-03,  1.51517981e-04,
        -4.90440774e-03],
       [-2.81498651e-01,  3.87277172e+00, -6.13541996e-01,
        -8.63617769e-01, -9.14461006e-01,  4.72618999e-01,
         8.12317704e-02,  8.56284005e-02,  4.93318881e-02,
         2.18977617e-02],
       [ 9.67673246e-03, -6.13541996e-01,  3.36048169e+00,
         5.30890176e-01,  2.37318481e-01, -1.71281010e-03,
         4.55218417e-02, -6.62658582e-04, -8.20000664e-

In [110]:
class MixedLogit(ChoiceModels):

    # Specify model here    
    model_config = {"Alternatives":
                    {"0": {"occupanc": 1, "route": 1},
                     "1": {"occupanc": 1, "route": 0},
                     "2": {"occupanc": 2, "route": 1},
                     "3": {"occupanc": 2, "route": 0},
                     "4": {"occupanc": 3, "route": 1},
                     "5": {"occupanc": 3, "route": 0}},
                    "Attributes":{'express_dummy':{'variable':'cons', 
                                                   'rule':{"0":1,"2":1,"4":1}},
                                  'hov2_dummy':{'variable':'cons', 
                                               'rule':{"2":1,"3":1}},
                                  "hov3_dummy":{'variable':'cons', 
                                                'rule':{"4":1,"5":1}},
                                  "price":{"variable": 'toll', 
                                           "rule": {"0":1,"2":1/2,"4":1/6}},
                                  "time": {"variable":"median", 
                                           "rule":{"0":1,"2":1,"4":1}}},
                    "Interactions":[('price', "high_income"), ('price', "med_income"),
                                    ("hov2_dummy", "householdsize"),
                                    ("hov3_dummy", "householdsize")],
                    "Mixedlogit":['price', 'time', 'hov2_dummy', 'hov3_dummy']}
    
    
    def __init__(self, path, file):
        df = super().load_data(path, file)
        res = super().create_choice_attributes(df, MixedLogit.model_config)
        self.df = res['data']
        self.X_list = res['var_names']
        self.y = self.df['y'].to_numpy()
        self.Xmat = self.df[self.X_list].to_numpy()

               
    def log_likelihood(self, para):
        df = self.df.copy()
        np.random.seed(123)
        b=randn(9)
        b=b*para
        xb = np.matmul(self.Xmat, b)
        xb = np.exp(xb)
        df['xb'] = xb.tolist()
       
        # group sum
        df['xbsum'] = df.groupby(['id'])["xb"].transform(lambda x: x.sum())
        df['log_likelihood'] = df['y']*np.log(df['xb'] / df['xbsum'])
        return (-1/len(df))* np.sum(df['log_likelihood'])
 
    def estimation(self, para):
        return super().optimization(self.log_likelihood, para)


In [111]:
#####################################
#Question 4
#It is reasonable to postulate that commuters are different in their preferences for travel time 
# and money costs. Try to investigate the heterogeneity among commuters based on the model 
# developed in step 2.   
#####################################

if __name__ == '__main__':
    path = r"iCloudDrive/Documents/Econs 514 (Metrics IV)/Assignment1"
    file = "assignment1_data.txt"
    
    ## estimating a MNL model
    mixl = MixedLogit(path, file)
    bini = np.zeros(len(mixl.X_list))
    res_mixl = mixl.estimation(bini)
    
print(" ")
print("Question 4: Y=Occup, Route")
print(" ")
print(res_mixl)


Optimization terminated successfully.
         Current function value: 0.221286
         Iterations: 59
         Function evaluations: 600
         Gradient evaluations: 60
 
Question 4: Y=Occup, Route
 
{'log_likelihood': -0.22128647589132702, 'Coefficients': array([  0.81801874,  -1.26127412, -10.47602385,   0.33129776,
         0.26944059,   0.28280532,  -0.08543017,   0.04215207,
         0.13872483]), 'Var_Cov': array([[ 1.80510421e+02,  3.21380960e+01,  1.34014478e+02,
        -2.94533718e+01,  4.51967151e+01, -2.97224793e+00,
         8.10485224e-02,  1.07038716e+01, -9.50978526e-01],
       [ 3.21380960e+01,  2.30969964e+02,  2.98922080e+02,
        -9.27046740e+00,  5.28053584e-01,  7.35194925e-01,
         6.12429646e-01,  1.30430516e+02, -1.43736913e+01],
       [ 1.34014478e+02,  2.98922080e+02,  7.08328986e+03,
        -6.76776274e+01, -3.39799058e+01, -1.21059437e+01,
         3.33182783e+00,  1.23884403e+02, -3.38422020e+02],
       [-2.94533718e+01, -9.27046740e+00, -6.